# Supervised classification

In the data.csv there are letters (uppercases and lowercases) and numbers, 28x28 pixels in a row format.

* First, you need to know which labels are which, meaning you need to visualize some data to realize which number labels represents a letter, or a number.

In [49]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [125]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [123]:
# Your code here
data=pd.read_csv('data_all.csv')
data.head(5)

,e,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,36.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,33.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
data.shape

(116322, 785)

In [44]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116322 entries, 0 to 116321
Columns: 750 entries, e to 775
dtypes: float64(750)
memory usage: 665.6 MB


In [37]:
# Parece estar llena de 0, veamos como se comportan los datos en total
col_vacias=[]

for column in list(data.columns):
    if sum(data[column]) >0:
        print(f'La columna {column} tiene: {sum(data[column])} datos')
    else:
        print(f'La columna {column} esta llena de 0s')
        col_vacias.append(column)

La columna e tiene: 2239512.0 datos
La columna 1 esta llena de 0s
La columna 2 esta llena de 0s
La columna 3 esta llena de 0s
La columna 4 esta llena de 0s
La columna 5 tiene: 1.0 datos
La columna 6 tiene: 126.0 datos
La columna 7 tiene: 249.0 datos
La columna 8 tiene: 250.0 datos
La columna 9 tiene: 208.0 datos
La columna 10 tiene: 82.0 datos
La columna 11 tiene: 3.0 datos
La columna 12 esta llena de 0s
La columna 13 tiene: 7.0 datos
La columna 14 tiene: 172.0 datos
La columna 15 tiene: 179.0 datos
La columna 16 tiene: 308.0 datos
La columna 17 tiene: 331.0 datos
La columna 18 tiene: 133.0 datos
La columna 19 tiene: 9.0 datos
La columna 20 tiene: 114.0 datos
La columna 21 tiene: 410.0 datos
La columna 22 tiene: 445.0 datos
La columna 23 tiene: 20.0 datos
La columna 24 tiene: 124.0 datos
La columna 25 tiene: 217.0 datos
La columna 26 tiene: 127.0 datos
La columna 27 tiene: 1.0 datos
La columna 28 esta llena de 0s
La columna 29 esta llena de 0s
La columna 30 tiene: 8.0 datos
La columna 

La columna 228 tiene: 1058806.0 datos
La columna 229 tiene: 2100613.0 datos
La columna 230 tiene: 3584639.0 datos
La columna 231 tiene: 5536421.0 datos
La columna 232 tiene: 7775248.0 datos
La columna 233 tiene: 9808915.0 datos
La columna 234 tiene: 11024442.0 datos
La columna 235 tiene: 11435144.0 datos
La columna 236 tiene: 11203385.0 datos
La columna 237 tiene: 10900304.0 datos
La columna 238 tiene: 10780815.0 datos
La columna 239 tiene: 10789415.0 datos
La columna 240 tiene: 10891545.0 datos
La columna 241 tiene: 11229420.0 datos
La columna 242 tiene: 11789532.0 datos
La columna 243 tiene: 12245726.0 datos
La columna 244 tiene: 12083889.0 datos
La columna 245 tiene: 11009684.0 datos
La columna 246 tiene: 9114880.0 datos
La columna 247 tiene: 6810181.0 datos
La columna 248 tiene: 4369049.0 datos
La columna 249 tiene: 2072018.0 datos
La columna 250 tiene: 529080.0 datos
La columna 251 tiene: 22898.0 datos
La columna 252 tiene: 418.0 datos
La columna 253 tiene: 424.0 datos
La columna 

La columna 444 tiene: 3237382.0 datos
La columna 445 tiene: 1499798.0 datos
La columna 446 tiene: 366839.0 datos
La columna 447 tiene: 13273.0 datos
La columna 448 tiene: 59.0 datos
La columna 449 tiene: 555.0 datos
La columna 450 tiene: 21658.0 datos
La columna 451 tiene: 614517.0 datos
La columna 452 tiene: 2557231.0 datos
La columna 453 tiene: 4986637.0 datos
La columna 454 tiene: 7281617.0 datos
La columna 455 tiene: 8950022.0 datos
La columna 456 tiene: 9875134.0 datos
La columna 457 tiene: 10078394.0 datos
La columna 458 tiene: 9918168.0 datos
La columna 459 tiene: 9825737.0 datos
La columna 460 tiene: 10394061.0 datos
La columna 461 tiene: 11443244.0 datos
La columna 462 tiene: 12189417.0 datos
La columna 463 tiene: 12053359.0 datos
La columna 464 tiene: 11143117.0 datos
La columna 465 tiene: 10196499.0 datos
La columna 466 tiene: 9701130.0 datos
La columna 467 tiene: 9480440.0 datos
La columna 468 tiene: 9091595.0 datos
La columna 469 tiene: 8259378.0 datos
La columna 470 tiene

La columna 667 tiene: 1697240.0 datos
La columna 668 tiene: 1004139.0 datos
La columna 669 tiene: 449542.0 datos
La columna 670 tiene: 107170.0 datos
La columna 671 tiene: 3191.0 datos
La columna 672 esta llena de 0s
La columna 673 esta llena de 0s
La columna 674 tiene: 7889.0 datos
La columna 675 tiene: 193330.0 datos
La columna 676 tiene: 767421.0 datos
La columna 677 tiene: 1605454.0 datos
La columna 678 tiene: 2721160.0 datos
La columna 679 tiene: 4118164.0 datos
La columna 680 tiene: 5619749.0 datos
La columna 681 tiene: 7014280.0 datos
La columna 682 tiene: 8076193.0 datos
La columna 683 tiene: 8725714.0 datos
La columna 684 tiene: 8922879.0 datos
La columna 685 tiene: 8703701.0 datos
La columna 686 tiene: 8104782.0 datos
La columna 687 tiene: 7193474.0 datos
La columna 688 tiene: 6080636.0 datos
La columna 689 tiene: 4956205.0 datos
La columna 690 tiene: 3927911.0 datos
La columna 691 tiene: 3075729.0 datos
La columna 692 tiene: 2350807.0 datos
La columna 693 tiene: 1755130.0 da

In [39]:
# Dropeamos las que no nos proporcionan información
data=data.drop(col_vacias,axis=1)
data.shape

(116322, 750)

In [45]:
data.e.describe()

count    116322.000000
mean         19.252695
std          17.817294
min           0.000000
25%           4.000000
50%          10.000000
75%          32.000000
max          61.000000
Name: e, dtype: float64

* Now, try to train a classifier model to predict the uppercases. Use every single model you know for classification.

* Try to do the same thing with numbers.

* Try to do the same thing with lowercases.

Creo que antes se trabajaba con otra base que tenía una columna de mayúsculas, minúsculas y números(binarios tal vez?)  
por lo que las instrucciones te indican que trabajes con las 3 opciones pero me enfocaré a la columna **e** de esta base

In [79]:
# Veamos como se distribuyen
posibles_resultados=sorted(data.e.unique())
len(posibles_resultados)

62

In [95]:
# Procedemos a separar nuestras label de clases

Dict_labels={}
count=1

for result in posibles_resultados:
    n="Class_"+str(result)
    Dict_labels[n]=np.array(X[y==result])
    count=count+1

## Árboles de decisión

In [112]:
X=data.drop('e',axis=1)
y=data['e']

# Hacemos nuestra partición
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.25,random_state=5)

# Corremos el modelo
params={'random_state':0,'max_depth':4}
classifier = DecisionTreeClassifier(**params)
classifier.fit(X_train, y_train)

# Y su predicción
y_test_pred = classifier.predict(X_test)

In [119]:
# Métricas del arbol de decisión
class_names = list(Dict_labels.keys())

print('\n'+"*"*60)
print('\n Classifier performance on Training dataset\n')
print(classification_report(y_train,classifier.predict(X_train),target_names=class_names,zero_division=0))
print('\n'+"*"*60)
print('\n Classifier performance on Test dataset\n')
print(classification_report(y_test,y_test_pred,target_names=class_names,zero_division=0))

# Precisision: Verdaderos positivos, porcentaje de datos que fueron correctamente predichos
# Recall: Cuantos items de los que seleccionaste fueron importantes y que realmente pertenecen a la clase
# f1 score: pondera ambos, Se usa cuando nos interesa tanto precission como recall
#          Si f1 es 0, significa que tu entrenamiento fue un desastre
#          Si f1 es 1 -> fue muy acertado



************************************************************

 Classifier performance on Training dataset

              precision    recall  f1-score   support

   Class_0.0       0.34      0.74      0.47      4370
   Class_1.0       0.48      0.84      0.61      4756
   Class_2.0       0.16      0.23      0.19      4418
   Class_3.0       0.28      0.53      0.36      4459
   Class_4.0       0.28      0.40      0.33      4137
   Class_5.0       0.51      0.48      0.50      3848
   Class_6.0       0.48      0.76      0.58      4373
   Class_7.0       0.60      0.74      0.66      4674
   Class_8.0       0.19      0.49      0.27      4212
   Class_9.0       0.44      0.75      0.55      4219
  Class_10.0       0.00      0.00      0.00       818
  Class_11.0       0.00      0.00      0.00       487
  Class_12.0       0.00      0.00      0.00      1296
  Class_13.0       0.00      0.00      0.00       574
  Class_14.0       0.00      0.00      0.00       646
  Class_15.0       0.00    

In [ ]:
# Salió medio mal :c la accuracy es muy baja y muchas variables quedaron en 0

In [128]:
# Confusion matrix 
y_pred_confussion = classifier.predict(X_test)
confusion_matrix(y_test,y_pred_confussion)

array([[1037,    1,   20, ...,    0,    0,    0],
       [   0, 1312,   21, ...,    0,    0,    0],
       [ 212,   16,  306, ...,    0,    0,    0],
       ...,
       [   0,    2,   27, ...,    0,    0,    0],
       [   0,    3,    2, ...,    0,    0,    0],
       [   0,    1,   43, ...,    0,    0,    0]], dtype=int64)

## Random Forest Classifier

In [129]:
from sklearn.ensemble import RandomForestClassifier

In [131]:
# Los estimatores son árboles 
# max_depth = 4 es la profundidad

# Corremos el modelo
classifier= RandomForestClassifier(n_estimators=100,max_depth=4, random_state=0)
classifier.fit(X_train, y_train)

# Y su predicción
y_test_pred = classifier.predict(X_test)

In [132]:
# Veamos las métricas esta vez
class_names = list(Dict_labels.keys())

print('\n'+"*"*60)
print('\n Classifier performance on Training dataset\n')
print(classification_report(y_train,classifier.predict(X_train),target_names=class_names,zero_division=0))
print('\n'+"*"*60)
print('\n Classifier performance on Test dataset\n')
print(classification_report(y_test,y_test_pred,target_names=class_names,zero_division=0))


************************************************************

 Classifier performance on Training dataset

              precision    recall  f1-score   support

   Class_0.0       0.37      0.91      0.53      4370
   Class_1.0       0.38      0.97      0.55      4756
   Class_2.0       0.51      0.78      0.62      4418
   Class_3.0       0.53      0.83      0.64      4459
   Class_4.0       0.40      0.80      0.53      4137
   Class_5.0       0.65      0.55      0.60      3848
   Class_6.0       0.48      0.89      0.62      4373
   Class_7.0       0.57      0.88      0.70      4674
   Class_8.0       0.42      0.50      0.46      4212
   Class_9.0       0.47      0.85      0.60      4219
  Class_10.0       0.00      0.00      0.00       818
  Class_11.0       0.00      0.00      0.00       487
  Class_12.0       1.00      0.00      0.00      1296
  Class_13.0       0.00      0.00      0.00       574
  Class_14.0       0.00      0.00      0.00       646
  Class_15.0       0.00    

In [ ]:
# Se incrementó nuestra accuracy a .45 :) pero aún no es muy buena

## KNN (K- Nearest Neighbor)


In [133]:
from sklearn import neighbors, datasets

In [137]:
# Corremos el modelo
num_neighbors = 10

classifier = neighbors.KNeighborsClassifier(num_neighbors, weights='distance')
classifier.fit(X,y)

KNeighborsClassifier(n_neighbors=10, weights='distance')

In [138]:
# Y la predicción
y_test_pred=classifier.predict(X_test)

## SVM


In [135]:
from sklearn import preprocessing
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsOneClassifier

In [136]:
# Corremos el modelo
classifier = OneVsOneClassifier(LinearSVC(random_state=0,max_iter=5000))
classifier.fit(X_train,y_train)

# Y la predicción
y_test_pred=classifier.predict(X_test)

# Calculemos el accuracy y recall ponderado
f1 = cross_val_score(classifier,X,y, scoring='f1_weighted',cv=3)
f1.mean()

C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

...

C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "
C:\Users\sarun\Anaconda3\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations

0.6854110873213527